# 🧪 CollapseGrammarOptimizer Submission Demo

This notebook demonstrates CollapseGrammarOptimizer_vGH1.0 under multiple stress scenarios.
It compares the performance against Adam, SGD, and RMSprop on synthetic collapse traces.


In [ ]:
import torch
import torch.nn as nn
import numpy as np
from torch.optim import Adam, SGD, RMSprop
from extreme_scenarios import generate_batch_traces
from CollapseGrammarOptimizer_vGH1 import CollapseGrammarOptimizer_vGH1
import matplotlib.pyplot as plt

In [ ]:
def run_optimizer_trace(optimizer_cls, mode, label):
    model = nn.Sequential(nn.Linear(100, 64), nn.ReLU(), nn.Linear(64, 1))
    traces = generate_batch_traces(mode, batch=32)
    targets = torch.ones((32, 1))
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optimizer_cls(model.parameters(), lr=1e-3)
    losses = []
    for epoch in range(20):
        output = model(traces)
        loss = criterion(output, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    return label, losses

In [ ]:
modes = ['vanishing_gradient', 'nan_divergence', 'chaotic_gradient', 
          'adversarial_spike', 'staircase_explosion', 'multi_modal_noise', 
          'plateau_burst', 'entropy_pulse']

results = {}

for mode in modes:
    print(f"Running trace: {mode}")
    results[mode] = []
    for opt_name, opt_class in {
        'GH': CollapseGrammarOptimizer_vGH1,
        'Adam': Adam,
        'SGD': SGD,
        'RMSprop': RMSprop
    }.items():
        label, losses = run_optimizer_trace(lambda p: opt_class(p, lr=1e-3), mode, opt_name)
        results[mode].append((label, losses))

In [ ]:
for mode, res in results.items():
    plt.figure(figsize=(10,6))
    for label, losses in res:
        plt.plot(losses, label=label)
    plt.title(f"Trace: {mode}")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend()
    plt.grid(True)
    plt.show()